In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from ydc.tools import import_data, distances
from supercats import add_supercats
from simplekml import Kml, Style
from colorsys import hsv_to_rgb

In [3]:
df = import_data.import_businesses()
cells = distances.CellCollection(15, df)

In [4]:
cell_copy = cells.to_dict()

In [5]:
(supercats_frame, names) = add_supercats(import_data.import_businesses())

In [6]:
cat_cells = {}
for col_name, col in cell_copy.items():
    for row_name, cell in col.items():
        cats = {}
        for business in cell:
            supercat = supercats_frame.iloc[business['index']]['super category']
            cats[supercat] = cats.get(supercat, 0) + 1
        if len(cats) != 0:
            df = pd.DataFrame(cats, index=['count']).transpose()
            max_cat = df['count'].idxmax()
            density = df.loc[max_cat]['count'] / df['count'].sum()
            # store into new dict
            if col_name not in cat_cells.keys():
                cat_cells[col_name] = {}
            cat_cells[col_name][row_name] = {'cat': int(max_cat), 'density': density}

In [46]:
densities = pd.DataFrame(cat_cells)

In [9]:
city= 'Edinburgh'

businesses = import_data.import_businesses()
businesses_in_city = businesses[businesses['city']==city]

def format_color(rgb_tuple, alpha):
    """ convert an (R, G, B) tuple to #RRGGBBal """
    rgb_tuple = (rgb_tuple[0]*255, rgb_tuple[1]*255, rgb_tuple[2]*255, alpha*255)
    hexcolor = '%02x%02x%02x%02x' % rgb_tuple
    return hexcolor

def make_polygon(folder, lowerleft, upperright, cell):
    pol = folder[cell['cat']].newpolygon(name='A Polygon')
    pol.extrude = 1
    coords = [lowerleft, (lowerleft[0], upperright[1]), upperright, (upperright[0], lowerleft[1]), lowerleft]
    pol.outerboundaryis = coords
    pol.style.polystyle.color = format_color(hsv_to_rgb(cell['cat']/len(names), 1, 1), cell['density'])
    
# Prepare KML
kml = Kml()

#Create subfolder for every category and save them in dictionary
subfolders = {key: kml.newfolder(name=names[key]) for key in names}

# Get boundaries for City
min_x = cells.get_cell(businesses.iloc[businesses_in_city['longitude'].idxmin()])[0]
max_x = cells.get_cell(businesses.iloc[businesses_in_city['longitude'].idxmax()])[0]
min_y = cells.get_cell(businesses.iloc[businesses_in_city['latitude'].idxmin()])[1]
max_y = cells.get_cell(businesses.iloc[businesses_in_city['latitude'].idxmax()])[1]
    
(longitudes, latitudes) = cells.get_borders()

for x in range(min_x, max_x):
    for y in range(min_y, max_y):
        if x in cat_cells and y in cat_cells[x]:
            make_polygon(subfolders, (longitudes[x], latitudes[y]), (longitudes[x+1], latitudes[y+1]), cat_cells[x][y])
            
kml.save("%s.kml" % city)

In [21]:
print("{} is near {}?".format(longitudes[min_x], businesses_in_city['longitude'].min()))

-3.3892822265625 is near -3.3913382?


In [23]:
latitudes[min_y]

55.8599853515625